In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import difflib

In [2]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [3]:
#import box score data
data = pd.read_csv('C:\\Users\gsteele\gamelogsABA.csv',na_values = ' ')
df = pd.DataFrame(data = data)
df.insert(loc = 0,column = 'game_id', value = df['M'].astype(str)+df['D'].astype(str)
          +df['Season'].astype(str).str.slice(1,5)+df['Season'].astype(str).str.slice(6,8)
          +df['team_1'].astype(str)+df['team_2'].astype(str))
df.insert(loc = 3,column = 'Name',value = df['first_name']+' '+df['last_name'])
df.insert(loc = 1,column = 'playerid',value = df['first_name']+df['last_name']+(df['Season']).astype(str))

#Clean NaNs for Rookie column
df['Rookie'] = df['Rookie'].fillna(value = 'No')

In [4]:
print(df)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
#import season totals, averages
season_data = pd.read_csv('C:\\Users\gsteele\ABAseasonstats.csv',low_memory = False,na_values = ' ')
season_frame = pd.DataFrame(data = season_data)
season_frame.set_index('League')
season = season_frame[season_frame.League == 'ABA']
season.insert(loc = 1,column  = 'playerid', 
value = season['firstname']+season['lastname']+season['Season'].astype(str))
seasons = season.fillna(value = 0.0)

In [6]:
def fuzzy_merge(df1, df2, left_on, right_on, how='inner', cutoff=0.9):
    df_other= seasons.copy()
    df_other[left_on] = [get_closest_match(x, df[left_on], cutoff) 
                         for x in df_other[right_on]]
    return df1.merge(df_other, on=left_on, how=how)

def get_closest_match(x, other, cutoff):
    matches = difflib.get_close_matches(x, other, cutoff=cutoff)
    return matches[0] if matches else x

In [7]:
aba = fuzzy_merge(df,seasons,left_on = 'playerid',right_on = 'playerid')
print(len(df))
print(len(aba))

76812
76810


In [8]:
#aba.to_csv("C:\\Users\gsteele\\aba_joined")

In [9]:
#aba = pd.read_csv("C:\\Users\gsteele\\aba_joined",low_memory = False)

In [10]:
#Calculating sums from game records

aba.insert(loc = 92, column = 'sum_pts', value = (aba[(aba['Type'] == 'Reg')].groupby(['Season_x','playerid'])['PTS'].transform("sum")))
aba.insert(loc = 93, column = 'sum_fgm', value = (aba[(aba['Type'] == 'Reg')].groupby(['Season_x','playerid'])['FGM'].transform("sum")))
aba.insert(loc = 94, column = 'sum_fga', value = (aba[(aba['Type'] == 'Reg')].groupby(['Season_x','playerid'])['FGA'].transform("sum")))
aba.insert(loc = 95, column = 'sum_fg3m', value = (aba[(aba['Type'] == 'Reg')].groupby(['Season_x','playerid'])['3FGM'].transform("sum")))
aba.insert(loc = 96, column = 'sum_fg3a', value = (aba[(aba['Type'] == 'Reg')].groupby(['Season_x','playerid'])['3FGA'].transform("sum")))
aba.insert(loc = 97, column = 'sum_ftm', value = (aba[(aba['Type'] == 'Reg')].groupby(['Season_x','playerid'])['FTM'].transform("sum")))
aba.insert(loc = 98, column = 'sum_fta', value = (aba[(aba['Type'] == 'Reg')].groupby(['Season_x','playerid'])['FTA'].transform("sum")))
aba.insert(loc = 99, column = 'sum_oreb', value = (aba[(aba['Type'] == 'Reg')].groupby(['Season_x','playerid'])['OFF'].transform("sum")))
aba.insert(loc = 100, column = 'sum_dreb', value = (aba[(aba['Type'] == 'Reg')].groupby(['Season_x','playerid'])['DEF'].transform("sum")))
aba.insert(loc = 101, column = 'sum_reb', value = (aba[(aba['Type'] == 'Reg')].groupby(['Season_x','playerid'])['REB'].transform("sum")))
aba.insert(loc = 102, column = 'sum_ast', value = (aba[(aba['Type'] == 'Reg')].groupby(['Season_x','playerid'])['AST'].transform("sum")))
aba.insert(loc = 103, column = 'sum_tov', value = (aba[(aba['Type'] == 'Reg')].groupby(['Season_x','playerid'])['TO'].transform("sum")))
aba.insert(loc = 104, column = 'sum_stl', value = (aba[(aba['Type'] == 'Reg')].groupby(['Season_x','playerid'])['STL'].transform("sum")))
aba.insert(loc = 105, column = 'sum_blk', value = (aba[(aba['Type'] == 'Reg')].groupby(['Season_x','playerid'])['BLK'].transform("sum")))
aba.insert(loc = 106, column = 'sum_pf', value = (aba[(aba['Type'] == 'Reg')].groupby(['Season_x','playerid'])['PF'].transform("sum")))

In [11]:
len(aba[(aba['points'] == 0) & (aba['sum_pts'] != 0)])

0

In [12]:
aba.insert(loc = 107, column = 'res_pts', 
            value = (aba['points'].str.replace(',','').astype(float) 
               - aba['sum_pts']))
aba.insert(loc = 107, column = 'res_fgm', 
            value = (aba['fieldgoalsmad'].str.replace(',','')
                     .astype(float)
                - aba['sum_fgm']))
aba.insert(loc = 107, column = 'res_fga', 
           value = (aba['fieldgoalsatt'].str.replace(',','').astype(float)
                - aba['sum_fga']))
    #AttributeError
aba.insert(loc = 107, column = 'res_fg3m', 
           value = (aba['threesmade']
                - aba['sum_fg3m']))
aba.insert(loc = 107, column = 'res_fg3a', 
           value = (aba['threesatt'].str.replace(',','').astype(float)
                - aba['sum_fg3a']))
aba.insert(loc = 107, column = 'res_ftm', 
           value = (aba['freethrowsmade'].astype(float)
                - aba['sum_ftm']))
aba.insert(loc = 107, column = 'res_fta', 
           value = (aba['freethrowsatt'].str.replace(',','').astype(float)
                - aba['sum_fta']))
    #Attribute Error
aba.insert(loc = 107, column = 'res_oreb', 
           value = (aba['offreb']
                - aba['sum_oreb'])) 
aba.insert(loc = 107, column = 'res_dreb', 
           value = (aba['defreb'].str.replace(',','').astype(float)
                - aba['sum_dreb']))
aba.insert(loc = 107, column = 'res_reb', 
           value = (aba['totalrebounds'].str.replace(',','').astype(float)
                - aba['sum_reb']))
aba.insert(loc = 107, column = 'res_ast', 
           value = (aba['assists'].str.replace(',','').astype(float)
                - aba['sum_ast']))
    #Attribute Error
aba.insert(loc = 107, column = 'res_tov', 
           value = (aba['turnovers'])
              - aba['sum_tov'])
aba.insert(loc = 107, column = 'res_pf', 
           value = (aba['fouls']
                - aba['sum_pf']))


In [13]:
aba = aba.apply(lambda x: np.isnull if isinstance(x, str) and (x.isspace() or not x) else x)
aba.insert(loc = 107, column = 'res_stl', value = (aba['sum_stl'] - aba['steals']))
aba.insert(loc = 107, column = 'res_blk', value = (aba['sum_blk'] - aba['blocks']))

In [14]:
len(aba[aba['steals'] == ' '])

C:\Users\gsteele\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


0

In [15]:
#Counting number of null records for each player-season

aba['tov_null'] = aba.TO.isnull()
aba['tov_null'] = aba.groupby(['Season_x','playerid'])['tov_null'].transform('sum').astype(int)

aba['pts_null'] = aba.PTS.isnull()
aba['pts_null'] = aba.groupby(['Season_x','playerid'])['pts_null'].transform('sum').astype(int)

aba['fgm_null'] = aba.FGM.isnull()
aba['fgm_null'] = aba.groupby(['Season_x','playerid'])['fgm_null'].transform('sum').astype(int)

aba['fga_null'] = aba.FGA.isnull()
aba['fga_null'] = aba.groupby(['Season_x','playerid'])['fga_null'].transform('sum').astype(int)

aba['3fgm_null'] = aba['3FGM'].isnull()
aba['3fgm_null'] = aba.groupby(['Season_x','playerid'])['3fgm_null'].transform('sum').astype(int)

aba['3fga_null'] = aba['3FGA'].isnull()
aba['3fga_null'] = aba.groupby(['Season_x','playerid'])['3fga_null'].transform('sum').astype(int)

aba['ftm_null'] = aba.FTM.isnull()
aba['ftm_null'] = aba.groupby(['Season_x','playerid'])['ftm_null'].transform('sum').astype(int)

aba['fta_null'] = aba.FTA.isnull()
aba['fta_null'] = aba.groupby(['Season_x','playerid'])['fta_null'].transform('sum').astype(int)

aba['oreb_null'] = aba.OFF.isnull()
aba['oreb_null'] = aba.groupby(['Season_x','playerid'])['oreb_null'].transform('sum').astype(int)

aba['dreb_null'] = aba.DEF.isnull()
aba['dreb_null'] = aba.groupby(['Season_x','playerid'])['dreb_null'].transform('sum').astype(int)

aba['reb_null'] = aba.REB.isnull()
aba['reb_null'] = aba.groupby(['Season_x','playerid'])['reb_null'].transform('sum').astype(int)

aba['ast_null'] = aba.AST.isnull()
aba['ast_null'] = aba.groupby(['Season_x','playerid'])['ast_null'].transform('sum').astype(int)

aba['stl_null'] = aba.STL.isnull()
aba['stl_null'] = aba.groupby(['Season_x','playerid'])['stl_null'].transform('sum').astype(int)

aba['blk_null'] = aba.BLK.isnull()
aba['blk_null'] = aba.groupby(['Season_x','playerid'])['blk_null'].transform('sum').astype(int)

aba['pf_null'] = aba.PF.isnull()
aba['pf_null'] = aba.groupby(['Season_x','playerid'])['pf_null'].transform('sum').astype(int)

aba['min_null'] = aba.MIN.isnull()
aba['min_null'] = aba.groupby(['Season_x','playerid'])['min_null'].transform('sum').astype(int)

In [16]:
#Finding average values for null records

aba['tov_where_null'] = (aba['res_tov']/aba['tov_null'])
aba['pts_where_null'] = (aba['res_pts']/aba['pts_null'])
aba['fgm_where_null'] = (aba['res_fgm']/aba['fgm_null'])
aba['fga_where_null'] = (aba['res_fga']/aba['fga_null'])
aba['3fgm_where_null'] = (aba['res_fg3m']/aba['3fgm_null'])
aba['3fga_where_null'] = (aba['res_fg3a']/aba['3fga_null'])
aba['ftm_where_null'] = (aba['res_ftm']/aba['ftm_null'])
aba['fta_where_null'] = (aba['res_fta']/aba['fta_null'])
aba['oreb_where_null'] = (aba['res_oreb']/aba['oreb_null'])
aba['dreb_where_null'] = (aba['res_dreb']/aba['dreb_null'])
aba['reb_where_null'] = (aba['res_reb']/aba['reb_null'])
aba['ast_where_null'] = (aba['res_ast']/aba['ast_null'])
aba['stl_where_null'] = (aba['res_stl']/aba['stl_null'])
aba['blk_where_null'] = (aba['res_blk']/aba['blk_null'])
aba['pf_where_null'] = (aba['res_pf']/aba['pf_null'])

In [17]:
aba.isnull().sum()

game_id                     0
playerid                    0
first_name                  0
last_name                   0
Name                        0
position                    3
Rookie                      0
Type                        0
D                           0
M                           0
Y                           0
Season_x                    0
team_1                      0
team_2                      0
Score                       0
Unnamed: 12                 0
W/L                         0
OT                      72340
G_x                         0
GS_x                    62898
MIN                     39246
FGM                         0
FGA                     25894
FG_Percentage           27307
3FGM                        0
3FGA                    46549
3FG_Percentage          66741
FTM                         0
FTA                         0
FT_Percentage           19520
OFF                     71734
DEF                     71734
REB                     24650
AST       

In [18]:
#Filling null records with average values for nulls

aba['PTS'].fillna(aba['pts_where_null'],inplace = True)
aba['FGM'].fillna(aba['fgm_where_null'],inplace = True)
aba['FGA'].fillna(aba['fga_where_null'],inplace = True)
aba['3FGM'].fillna(aba['3fgm_where_null'],inplace = True)
aba['3FGA'].fillna(aba['3fga_where_null'],inplace = True)
aba['FTM'].fillna(aba['ftm_where_null'],inplace = True)
aba['FTA'].fillna(aba['fta_where_null'],inplace = True)
aba['OFF'].fillna(aba['oreb_where_null'],inplace = True)
aba['DEF'].fillna(aba['dreb_where_null'],inplace = True)
aba['REB'].fillna(aba['reb_where_null'],inplace = True)
aba['AST'].fillna(aba['ast_where_null'],inplace = True)
aba['TO'].fillna(aba['tov_where_null'],inplace = True)
aba['STL'].fillna(aba['stl_where_null'],inplace = True)
aba['BLK'].fillna(aba['blk_where_null'],inplace = True)
aba['PF'].fillna(aba['pf_where_null'],inplace = True)


aba.isnull().sum()

game_id                     0
playerid                    0
first_name                  0
last_name                   0
Name                        0
position                    3
Rookie                      0
Type                        0
D                           0
M                           0
Y                           0
Season_x                    0
team_1                      0
team_2                      0
Score                       0
Unnamed: 12                 0
W/L                         0
OT                      72340
G_x                         0
GS_x                    62898
MIN                     39246
FGM                         0
FGA                      1148
FG_Percentage           27307
3FGM                        0
3FGA                     2750
3FG_Percentage          66741
FTM                         0
FTA                         0
FT_Percentage           19520
OFF                      4977
DEF                     11773
REB                      1105
AST       

In [19]:
#aba.fillna(0,inplace = True)
#aba.isnull().sum()

In [20]:
aba.head(5)

,game_id,playerid,first_name,last_name,Name,position,Rookie,Type,D,M,Y,Season_x,team_1,team_2,Score,Unnamed: 12,W/L,OT,G_x,GS_x,MIN,FGM,FGA,FG_Percentage,3FGM,3FGA,3FG_Percentage,FTM,FTA,FT_Percentage,OFF,DEF,REB,AST,STL,BLK,TO,PF,PTS,Notes,League,firstname,lastname,pos,Rook,Champion,season_type,Season_y,Team(s),Games,Started,MP,fieldgoalsmad,fieldgoalsatt,fieldgoalpercentage,threesmade,threesatt,threepointpercentage,freethrowsmade,freethrowsatt,freethrowpercentage,offreb,defreb,totalrebounds,assists,steals,blocks,turnovers,fouls,points,Unnamed: 30,G_y,GS_y,mpg,fgmpg,fgapg,FG%,fg3mpg,fg3apg,3FG%,ftmpg,ftapm,FT%,orebpg,drebpg,rebpg,apg,spg,bpg,tovpg,fpg,ppg,sum_pts,sum_fgm,sum_fga,sum_fg3m,sum_fg3a,sum_ftm,sum_fta,sum_oreb,sum_dreb,sum_reb,sum_ast,sum_tov,sum_stl,sum_blk,sum_pf,res_blk,res_stl,res_pf,res_tov,res_ast,res_reb,res_dreb,res_oreb,res_fta,res_ftm,res_fg3a,res_fg3m,res_fga,res_fgm,res_pts,tov_null,pts_null,fgm_null,fga_null,3fgm_null,3fga_null,ftm_null,fta_null,oreb_null,dreb_null,reb_null,ast_null,stl_null,blk_null,pf_null,min_null,tov_where_null,pts_where_null,fgm_where_null,fga_where_null,3fgm_where_null,3fga_where_null,ftm_where_null,fta_where_null,oreb_where_null,dreb_where_null,reb_where_null,ast_where_null,stl_where_null,blk_where_null,pf_where_null
0,1013196768ANAOAK,BobBedell(1967-68),Bob,Bedell,Bob Bedell,F,rookie,Reg,13,10,1967,(1967-68),ANA,OAK,129,134,L,NaN,1,NaN,NaN,1,5.000000,20%,0,0.057971,NaN,1,2,50%,0.0,NaN,4.000000,1.114286,0.0,0.0,1.5,1.000000,3,NaN,ABA,Bob,Bedell,F,rookie,0,Regular Season,(1967-68),ANA,76,0.0,"1,492",325,736,44%,0.0,4,0%,142,190,75%,0.0,0,506,79,0.0,0.0,111.0,203.0,792,0.0,0.0,0.0,19.63,4.28,9.68,44%,0.0,0.05,0%,1.87,2.5,75%,0.0,0.0,6.66,1.04,0.0,0.0,1.46,2.67,10.42,792.0,325.0,137.0,0.0,0.0,142.0,190.0,0.0,0.0,115.0,1.0,0.0,0.0,0.0,104.0,0.0,0.0,99.0,111.0,78.0,391.0,NaN,0.0,0.0,0.0,4.0,0.0,599.0,0.0,0.0,74,0,0,54,0,69,0,0,72,72,56,70,74,74,34,69,1.5,NaN,NaN,11.092593,NaN,0.057971,NaN,NaN,0.0,NaN,6.982143,1.114286,0.0,0.0,2.911765
1,1015196768ANADEN,BobBedell(1967-68),Bob,Bedell,Bob Bedell,F,rookie,Reg,15,10,1967,(1967-68),ANA,DEN,105,110,L,NaN,1,NaN,NaN,1,11.092593,NaN,0,0.057971,NaN,0,0,NaN,0.0,NaN,6.982143,1.114286,0.0,0.0,1.5,2.911765,2,NaN,ABA,Bob,Bedell,F,rookie,0,Regular Season,(1967-68),ANA,76,0.0,"1,492",325,736,44%,0.0,4,0%,142,190,75%,0.0,0,506,79,0.0,0.0,111.0,203.0,792,0.0,0.0,0.0,19.63,4.28,9.68,44%,0.0,0.05,0%,1.87,2.5,75%,0.0,0.0,6.66,1.04,0.0,0.0,1.46,2.67,10.42,792.0,325.0,137.0,0.0,0.0,142.0,190.0,0.0,0.0,115.0,1.0,0.0,0.0,0.0,104.0,0.0,0.0,99.0,111.0,78.0,391.0,NaN,0.0,0.0,0.0,4.0,0.0,599.0,0.0,0.0,74,0,0,54,0,69,0,0,72,72,56,70,74,74,34,69,1.5,NaN,NaN,11.092593,NaN,0.057971,NaN,NaN,0.0,NaN,6.982143,1.114286,0.0,0.0,2.911765
2,1016196768ANADAL,BobBedell(1967-68),Bob,Bedell,Bob Bedell,F,rookie,Reg,16,10,1967,(1967-68),ANA,DAL,125,129,L,NaN,1,NaN,NaN,2,11.092593,NaN,0,0.057971,NaN,2,2,100%,0.0,NaN,6.982143,1.114286,0.0,0.0,1.5,2.911765,6,NaN,ABA,Bob,Bedell,F,rookie,0,Regular Season,(1967-68),ANA,76,0.0,"1,492",325,736,44%,0.0,4,0%,142,190,75%,0.0,0,506,79,0.0,0.0,111.0,203.0,792,0.0,0.0,0.0,19.63,4.28,9.68,44%,0.0,0.05,0%,1.87,2.5,75%,0.0,0.0,6.66,1.04,0.0,0.0,1.46,2.67,10.42,792.0,325.0,137.0,0.0,0.0,142.0,190.0,0.0,0.0,115.0,1.0,0.0,0.0,0.0,104.0,0.0,0.0,99.0,111.0,78.0,391.0,NaN,0.0,0.0,0.0,4.0,0.0,599.0,0.0,0.0,74,0,0,54,0,69,0,0,72,72,56,70,74,74,34,69,1.5,NaN,NaN,11.092593,NaN,0.057971,NaN,NaN,0.0,NaN,6.982143,1.114286,0.0,0.0,2.911765
3,1018196768ANAIND,BobBedell(1967-68),Bob,Bedell,Bob Bedell,F,rookie,Reg,18,10,1967,(1967-68),ANA,IND,103,106,L,NaN,1,NaN,NaN,0,0.000000,NaN,0,0.000000,NaN,0,0,NaN,0.0,NaN,6.982143,1.114286,0.0,0.0,1.5,1.000000,0,NaN,ABA,Bob,Bedell,F,rookie,0,Regular Season,(1967-68),ANA,76,0.0,"1,492",325,736,44%,0.0,4,0%,142,190,75%,0.0,0,506,79,0.0,0.0,111.0,203.0,792,0.0,0.0,0.0,19.63,4.28,9.68,44%,0.0,0.05,0%,1.87,2.5,75%,0.0,0.0,6.66,1.04,0.0,0.0,1.46,2.67,10.42,792.0,325.0,137.0,0.0,0.0,142.0,190.0,0.0,0.0,115.0,1.0,0.0,0.0,0.0,104.0,

In [21]:
aba = aba[aba['Type'] == 'Reg'].copy()

In [35]:
aba['fieldgoalsatt'] = aba['fieldgoalsatt'].str.replace(',','')
aba['freethrowsatt'] = aba['freethrowsatt'].str.replace(',','')
aba['points'] = aba['points'].str.replace(',','')

In [46]:
#Renaming columns, calculating eFG% and TS%
aba['player_id'] = aba['playerid']
aba['Season'] = aba['Season_x']
aba['team_abbreviation'] = aba['team_1']
aba['opponent'] = aba['team_2']
aba['team_pts'] = aba['Score']
aba['opponent_pts'] = aba['Unnamed: 12']
aba['MISSES'] = (aba['FGA'] - aba['FGM'])
aba['FG2M'] = (aba['FGM'] - aba['3FGM'])
aba['FG2A'] = (aba['FGA'] - aba['3FGA'])
aba['FG3M'] = aba['3FGM']
aba['FG3A'] = aba['3FGA']
aba['FG3Percentage'] = aba['3FGM'].astype(int)/aba['3FGA'].astype(int)
aba['FTPercentage'] = aba['FTM'].astype(int)/aba['FTA'].astype(int)
aba['FGPercentage'] = aba['FGM'].astype(int)/aba['FGA'].astype(int)
aba['FTMISSES'] = (aba['FTA'] - aba['FTM'])
aba['OREB'] = aba['OFF']
aba['DREB'] = aba['DEF']
aba['TOV'] = aba['TO']
aba['eFG'] = (((aba['PTS']-aba['FTM'])/aba['FGA'])/2)
aba['TS'] = ((aba['PTS']/(aba['FGA']+(0.44*(aba['FTA']))))/2)
aba['total_FGM'] = aba['fieldgoalsmad']
aba['total_FGA'] = aba['fieldgoalsatt']
aba['total_FGPERCENTAGE'] = (aba['fieldgoalsmad'].astype(int)
                             /aba['fieldgoalsatt'].astype(int))
aba['total_FG3M'] = aba['threesmade']
aba['total_FG3A'] = aba['threesatt']
aba['total_FG3PERCENTAGE'] = (aba['threesmade'].astype(int)
                             /aba['threesatt'].astype(int))
aba['total_FTM'] = aba['freethrowsmade']
aba['total_FTA'] = aba['freethrowsatt']
aba['total_FTPERCENTAGE'] = (aba['freethrowsmade'].astype(int)
                             /aba['freethrowsatt'].astype(int))
aba['total_OREB'] = aba['offreb']
aba['total_DREB'] = aba['defreb']
aba['total_REB'] = aba['totalrebounds']
aba['total_AST'] = aba['assists']
aba['total_STL'] = aba['steals']
aba['total_BLK'] = aba['blocks']
aba['total_TOV'] = aba['turnovers']
aba['total_PF'] = aba['fouls']
aba['total_PTS'] = aba['points']
aba['total_MISSES'] = aba['fieldgoalsatt'].astype(int) 
- aba['fieldgoalsmad'].str.strip(',').astype(int)
aba['total_FTMISSES'] = aba['freethrowsatt'].astype(int) 
- aba['freethrowsmade'].astype(int)
aba['total_eFG'] = (((aba['points'].astype(int)
                      -aba['freethrowsmade'].astype(int))
                     /aba['fieldgoalsatt'].astype(int))/2)
aba['total_TS'] = ((aba['points'].astype(int)/
                    (aba['fieldgoalsatt'].astype(int)
                     +(0.44*(aba['freethrowsatt'].astype(int)))))/2)
aba.head(5)

,game_id,playerid,first_name,last_name,Name,position,Rookie,Type,D,M,Y,Season_x,team_1,team_2,Score,Unnamed: 12,W/L,OT,G_x,GS_x,MIN,FGM,FGA,FG_Percentage,3FGM,3FGA,3FG_Percentage,FTM,FTA,FT_Percentage,OFF,DEF,REB,AST,STL,BLK,TO,PF,PTS,Notes,League,firstname,lastname,pos,Rook,Champion,season_type,Season_y,Team(s),Games,Started,MP,fieldgoalsmad,fieldgoalsatt,fieldgoalpercentage,threesmade,threesatt,threepointpercentage,freethrowsmade,freethrowsatt,freethrowpercentage,offreb,defreb,totalrebounds,assists,steals,blocks,turnovers,fouls,points,Unnamed: 30,G_y,GS_y,mpg,fgmpg,fgapg,FG%,fg3mpg,fg3apg,3FG%,ftmpg,ftapm,FT%,orebpg,drebpg,rebpg,apg,spg,bpg,tovpg,fpg,ppg,sum_pts,sum_fgm,sum_fga,sum_fg3m,sum_fg3a,sum_ftm,sum_fta,sum_oreb,sum_dreb,sum_reb,sum_ast,sum_tov,sum_stl,sum_blk,sum_pf,res_blk,res_stl,res_pf,res_tov,res_ast,res_reb,res_dreb,res_oreb,res_fta,res_ftm,res_fg3a,res_fg3m,res_fga,res_fgm,res_pts,tov_null,pts_null,fgm_null,fga_null,3fgm_null,3fga_null,ftm_null,fta_null,oreb_null,dreb_null,reb_null,ast_null,stl_null,blk_null,pf_null,min_null,tov_where_null,pts_where_null,fgm_where_null,fga_where_null,3fgm_where_null,3fga_where_null,ftm_where_null,fta_where_null,oreb_where_null,dreb_where_null,reb_where_null,ast_where_null,stl_where_null,blk_where_null,pf_where_null,player_id,Season,team_abbreviation,opponent,team_pts,opponent_pts,MISSES,FG2M,FG2A,FG3M,FG3A,FG3_Percentage,FTMISSES,OREB,DREB,TOV,eFG,TS,total_FGM,total_FGA,total_FGPERCENTAGE,total_FG3M,total_FG3A,total_FG3PERCENTAGE,total_FTM,total_FTA,total_FTPERCENTAGE,total_OREB,total_DREB,total_REB,total_AST,total_STL,total_BLK,total_TOV,total_PF,total_PTS,total_MISSES,total_FTMISSES,total_eFG,total_TS,FG3Percentage,FTPercentage,FGPercentage
0,1013196768ANAOAK,BobBedell(1967-68),Bob,Bedell,Bob Bedell,F,rookie,Reg,13,10,1967,(1967-68),ANA,OAK,129,134,L,NaN,1,NaN,NaN,1,5.000000,20%,0,0.057971,NaN,1,2,50%,0.0,NaN,4.000000,1.114286,0.0,0.0,1.5,1.000000,3,NaN,ABA,Bob,Bedell,F,rookie,0,Regular Season,(1967-68),ANA,76,0.0,"1,492",325,736,44%,0.0,4,0%,142,190,75%,0.0,0,506,79,0.0,0.0,111.0,203.0,792,0.0,0.0,0.0,19.63,4.28,9.68,44%,0.0,0.05,0%,1.87,2.5,75%,0.0,0.0,6.66,1.04,0.0,0.0,1.46,2.67,10.42,792.0,325.0,137.0,0.0,0.0,142.0,190.0,0.0,0.0,115.0,1.0,0.0,0.0,0.0,104.0,0.0,0.0,99.0,111.0,78.0,391.0,NaN,0.0,0.0,0.0,4.0,0.0,599.0,0.0,0.0,74,0,0,54,0,69,0,0,72,72,56,70,74,74,34,69,1.5,NaN,NaN,11.092593,NaN,0.057971,NaN,NaN,0.0,NaN,6.982143,1.114286,0.0,0.0,2.911765,BobBedell(1967-68),(1967-68),ANA,OAK,129,134,4.000000,1,4.942029,0,0.057971,NaN,1,0.0,NaN,1.5,0.200000,0.255102,325,736,0.441576,0.0,4,0.0,142,190,0.747368,0.0,0,506,79,0.0,0.0,111.0,203.0,792,736,190,0.441576,0.483163,NaN,0.5,0.200000
1,1015196768ANADEN,BobBedell(1967-68),Bob,Bedell,Bob Bedell,F,rookie,Reg,15,10,1967,(1967-68),ANA,DEN,105,110,L,NaN,1,NaN,NaN,1,11.092593,NaN,0,0.057971,NaN,0,0,NaN,0.0,NaN,6.982143,1.114286,0.0,0.0,1.5,2.911765,2,NaN,ABA,Bob,Bedell,F,rookie,0,Regular Season,(1967-68),ANA,76,0.0,"1,492",325,736,44%,0.0,4,0%,142,190,75%,0.0,0,506,79,0.0,0.0,111.0,203.0,792,0.0,0.0,0.0,19.63,4.28,9.68,44%,0.0,0.05,0%,1.87,2.5,75%,0.0,0.0,6.66,1.04,0.0,0.0,1.46,2.67,10.42,792.0,325.0,137.0,0.0,0.0,142.0,190.0,0.0,0.0,115.0,1.0,0.0,0.0,0.0,104.0,0.0,0.0,99.0,111.0,78.0,391.0,NaN,0.0,0.0,0.0,4.0,0.0,599.0,0.0,0.0,74,0,0,54,0,69,0,0,72,72,56,70,74,74,34,69,1.5,NaN,NaN,11.092593,NaN,0.057971,NaN,NaN,0.0,NaN,6.982143,1.114286,0.0,0.0,2.911765,BobBedell(1967-68),(1967-68),ANA,DEN,105,110,10.092593,1,11.034622,0,0.057971,NaN,0,0.0,NaN,1.5,0.090150,0.090150,325,736,0.441576,0.0,4,0.0,142,190,0.747368,0.0,0,506,79,0.0,0.0,111.0,203.0,792,736,190,0.441576,0.483163,NaN,NaN,0.090909
2,1016196768ANADAL,BobBedell(1967-68),Bob,Bedell,Bob Bedell,F,rookie,Reg,16,10,1967,(1967-68),ANA,DAL,125,129,L,NaN,1,NaN,NaN,2,11.092593,NaN,0,0.057971,NaN,2,2,100%,0.0,NaN,6.982143,1.114286,0.0,0.0,1.5,2.911765,6,NaN,ABA,Bob,Bedell,F,rookie,0,Regular Season,(1967-68),ANA,76,0.0,"1,492",325,736,44%,0.0,4,0%,142,190,75%,0.0,0,506,79,0.0,0.0,111.0,

In [47]:
aba_box_scores = aba[['game_id','player_id','Name','first_name','last_name','position',
           'Rookie','D','M','Y','Season','team_abbreviation','opponent','team_pts',
           'opponent_pts','MIN','PTS','FGM','FGA','MISSES','FGPercentage',
          'FG2M','FG2A','FG3M','FG3A','FG3Percentage','FTM','FTA','FTMISSES',
          'FTPercentage','eFG','TS','OREB','DREB','REB','AST','TOV','STL','BLK',
            'PF','season_type','W/L','OT']].copy()

In [49]:
aba_season_totals = aba[['player_id','Name','first_name','last_name','position',
                        'Rookie','Season','Team(s)','MP','Games',
                        'Started','total_PTS','total_FGM','total_FGA',
                         'total_FGPERCENTAGE','total_MISSES','total_FG3M',
                         'total_FG3A','total_FG3PERCENTAGE','total_FTM',
                         'total_FTA','total_FTMISSES','total_FTPERCENTAGE',
                         'total_eFG','total_TS',
                         'total_OREB','total_DREB','total_REB','total_AST',
                         'total_TOV','total_STL','total_BLK','total_PF',
                         'season_type','Champion']].copy()

In [52]:
aba_box_scores.to_csv("C:\\Users\gsteele\\aba_box_scores.csv")
aba_season_totals.to_csv("C:\\Users\gsteele\\aba_season_totals.csv")